In [1]:
import numpy as np
import pandas as pd
import joblib
import pickle as pkl
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### chargement de données

In [2]:
#X_train, X_test, y_train, y_test = joblib.load("data_split.pkl")

In [5]:
y_train = joblib.load("y_train.joblib")

In [6]:
X_train = joblib.load("X_train.joblib")

In [4]:
X_test = joblib.load("X_test.joblib")

In [5]:
y_test = joblib.load("y_test.joblib")

### gridsearch for SVC

In [6]:
# 解説5：グリッドサーチを設定する---------------------------------------------------
#clf=SVC(class_weight='balanced', random_state=42)
clf=SVC(random_state=42)

param_range1=[0.001,0.01] #変化させるパラメータに設定する値たち
param_range2=[10,100,1000]
hyper_param_grid=[{'C':param_range2,
             'kernel':['rbf'], 
             'gamma':param_range1}] #Cとカーネルとgammaを変化させて最適化させる

clf_grid_search = GridSearchCV(
    estimator = clf, 
    param_grid = hyper_param_grid, 
    cv=3, 
    return_train_score=True,
    verbose=2)

clf_grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time=25.9min
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time=27.4min
[CV] END ......................C=10, gamma=0.001, kernel=rbf; total time=28.7min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=87.8min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=87.0min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time=90.7min
[CV] END .....................C=100, gamma=0.001, kernel=rbf; total time=48.8min
[CV] END .....................C=100, gamma=0.001, kernel=rbf; total time=48.0min
[CV] END .....................C=100, gamma=0.001, kernel=rbf; total time=48.6min
[CV] END ......................C=100, gamma=0.01, kernel=rbf; total time=88.2min
[CV] END ......................C=100, gamma=0.01, kernel=rbf; total time=99.7min
[CV] END ......................C=100, gamma=0.01,

GridSearchCV(cv=3, estimator=SVC(random_state=42),
             param_grid=[{'C': [10, 100, 1000], 'gamma': [0.001, 0.01],
                          'kernel': ['rbf']}],
             return_train_score=True, verbose=2)

In [12]:
results=pd.DataFrame(clf_grid_search.cv_results_)
results.T

,0,1,2,3,4,5
mean_fit_time,935.323572,4193.65592,2312.319777,4422.959512,2425.164634,4290.382791
std_fit_time,54.219685,59.723849,25.393948,299.220161,37.253767,57.268851
mean_score_time,704.999566,1118.76803,596.532618,1118.591475,580.343967,1080.23657
std_score_time,30.548035,38.3213,36.516442,16.730515,26.454818,31.113071
param_C,10,10,100,100,1000,1000
param_gamma,0.001,0.01,0.001,0.01,0.001,0.01
param_kernel,rbf,rbf,rbf,rbf,rbf,rbf
params,"{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}","{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}","{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}","{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}","{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}","{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}"
split0_test_score,0.883489,0.883849,0.865011,0.884569,0.861411,0.884569
split1_test_score,0.879275,0.880475,0.866075,0.881195,0.860314,0.881195


In [7]:
# to see the best hyperparameters

clf_grid_search.best_params_

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

In [9]:
# to see the best validation score
clf_grid_search.best_score_

0.8833599516286866

In [15]:
y_train_pred = clf_grid_search.predict(X_train)

In [16]:
accuracy_train = accuracy_score(y_train, y_train_pred)
print(f'train/validation accuracy: {accuracy_train}')

train/validation accuracy: 1.0


In [10]:
# to see the best hyperparameters
clf_svc = clf_grid_search.best_estimator_
clf_svc

SVC(C=100, gamma=0.01, random_state=42)

modèle avec best hyperparameters est over fitting

    train score(accuracy): 1.0
    validation score(accuracy) : 0.8833599516286866

donc c'est mieux de prendre le modèle avec test_rank_score=4

    train score(accuracy): 0.95932
    validation score(accuracy) : 0.88252
    train score(accuracy) - validation score(accuracy) = 0.0768

    SVC(C=10, gamma=0.001, kernel=rbf, random_state=42)

### Focus sur le meilleur modele

#### matrix de confusion

In [18]:
#création de matrix de confusion
cm_train = confusion_matrix(y_train, y_train_pred)

In [19]:
cm_train

array([[12500,     0],
       [    0, 12500]], dtype=int64)

### save the best model avec joblib

In [11]:
%%time
# 構築したモデルの保存
# best_estimator_ over fitting
filename = 'clf_svc.pkl'
joblib.dump(clf_svc,filename)

Wall time: 965 ms


['clf_svc.pkl']

le modéle non over fitting

In [3]:
clf_svc_best = SVC(C=10, gamma=0.001, kernel='rbf', random_state=42)

In [7]:
clf_svc_best.fit(X_train, y_train)

SVC(C=10, gamma=0.001, random_state=42)

In [8]:
%%time
# 構築したモデルの保存
filename = 'clf_svc_best.pkl'
joblib.dump(clf_svc_best,filename)

Wall time: 679 ms


['clf_svc_best.pkl']

https://www.salesanalytics.co.jp/datascience/datascience044/